<a href="https://colab.research.google.com/github/parker-group/Kinshasa_EO/blob/main/GEE_ERA5_Export_Kinshasa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install earthengine-api geemap --quiet
import ee
import geemap
from datetime import datetime
from dateutil.relativedelta import relativedelta
import time

ee.Authenticate()
ee.Initialize(project='ee-salempar')

roi = ee.FeatureCollection("projects/ee-salempar/assets/Kinshasa_AiresSelect2")

start_date = datetime(2022, 1, 1)
end_date = datetime(2023, 12, 31)

def export_era5_rasters(band_name, label_prefix, scale, folder, reducer):
    current = start_date
    while current <= end_date:
        next_month = current + relativedelta(months=1)
        label = current.strftime("%Y_%m")

        ee_start = ee.Date(current.strftime("%Y-%m-%d"))
        ee_end = ee.Date(next_month.strftime("%Y-%m-%d"))

        image = (ee.ImageCollection("ECMWF/ERA5_LAND/MONTHLY_AGGR")
                 .filterDate(ee_start, ee_end)
                 .filterBounds(roi)
                 .select(band_name)
                 .reduce(reducer)
                 .clip(roi))

        if band_name == 'temperature_2m':
            image = image.subtract(273.15).rename(band_name)

        task = ee.batch.Export.image.toDrive(**{
            'image': image,
            'description': f"{label_prefix}_{label}",
            'folder': folder,
            'fileNamePrefix': f"{label_prefix}_{label}",
            'scale': scale,
            'region': roi.geometry(),
            'crs': 'EPSG:4326',
            'maxPixels': 1e13
        })
        task.start()
        print(f"✅ Started export: {label_prefix}_{label}")

        current = next_month

start_time = time.time()

export_era5_rasters('temperature_2m', 'ERA5_Temp', 1000, 'GEE_Kinshasa', ee.Reducer.mean())
export_era5_rasters('total_precipitation_sum', 'ERA5_Precip', 1000, 'GEE_Kinshasa', ee.Reducer.sum())

end_time = time.time()
elapsed_minutes = (end_time - start_time) / 60
print(f"⏱️ Total script runtime: {elapsed_minutes:.2f} minutes")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 17.1 MB/s eta 0:00:00
✅ Started export: ERA5_Temp_2022_01
✅ Started export: ERA5_Temp_2022_02
✅ Started export: ERA5_Temp_2022_03
✅ Started export: ERA5_Temp_2022_04
✅ Started export: ERA5_Temp_2022_05
✅ Started export: ERA5_Temp_2022_06
✅ Started export: ERA5_Temp_2022_07
✅ Started export: ERA5_Temp_2022_08
✅ Started export: ERA5_Temp_2022_09
✅ Started export: ERA5_Temp_2022_10
✅ Started export: ERA5_Temp_2022_11
✅ Started export: ERA5_Temp_2022_12
✅ Started export: ERA5_Temp_2023_01
✅ Started export: ERA5_Temp_2023_02
✅ Started export: ERA5_Temp_2023_03
✅ Started export: ERA5_Temp_2023_04
✅ Started export: ERA5_Temp_2023_05
✅ Started export: ERA5_Temp_2023_06
✅ Started export: ERA5_Temp_2023_07
✅ Started export: ERA5_Temp_2023_08
✅ Started export: ERA5_Temp_2023_09
✅ Started export: ERA5_Temp_2023_10
✅ Started export: ERA5_Temp_2023_11
✅ Started export: ERA5_Temp_2023_12
✅ Started export: ERA5_Precip_2022_01
✅ Started export: ERA